In [1]:
%load_ext autoreload
%autoreload 2

### Leo un archivo simulado

In [9]:
from npfd.paths import raw_data_path, interim_data_path, processed_data_path

In [11]:
files = (raw_data_path / 'malte-uhma').glob('*2-*')
for file in files:
    print(file)

/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20191020.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20161105.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20191019.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20190425.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20180828.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20200129.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20180616.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20160223.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20170625.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20160524.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte-uhma/2-20190828.h5
/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/raw/malte

In [57]:
from npfd.data.dataset import Dataset

sds = Dataset(dataset_name='malte-uhma')

In [59]:
sds.name

'malte-uhma'

In [48]:
import pandas as pd

data_version = 2
date = '20181109'
filename = f"{data_version}-{date}.h5"
data_path = raw_data_path / 'malte-uhma'
size_dist_df = pd.read_hdf(data_path / file, key='obs/particle').resample('10T').mean() / 1e6


In [ ]:
def read_raw_simulations(params, test_size=0.1):
    """ Generate data files to be used by HTK

    Notes
    -----
    Data is resampeled to 10 minutes period

    """

    if not os.listdir(DATA_TEST_PATH):
        # Convert and split file into test.synth and train.synth
        for file in os.listdir(RAW_SIMULATION_DATA_PATH):
            if file.endswith('.h5') and file.startswith(params['data_version']):
                file_name = file[:-3]

                # Read in size distribution data
                size_dist_df = pd.read_hdf(RAW_SIMULATION_DATA_PATH + file, key='obs/particle').resample('10T').mean() / 1e6

                if params['normalize']:
                    # size_dist_df = pd.DataFrame(index=size_dist_df.index, data=cm3_to_dndlogdp(size_dist_df / 1e6))
                    size_dist_df = np.log10(
                        np.absolute(pd.DataFrame(index=size_dist_df.index, data=cm3_to_dndlogdp(size_dist_df)) + 10))
                else:
                    pass

                # Calculate labels
                if params['label_type'] == 'event-noevent':
                    labels = get_labels_ene(file, params)
                elif params['label_type'] == 'nccd':
                    labels = get_labels_nccd(file, params)

                # Split data, 90% for training, 10% for testing
                if np.random.rand() < test_size:
                    fo = os.path.join(interim_data_path, 'test.synth', file_name[2:])
                    fo_label = os.path.join(LABEL_TEST_PATH, file_name[2:])
                else:
                    fo = os.path.join(interim_data_path, 'train.synth', file_name[2:])
                    fo_label = os.path.join(LABEL_TRAIN_PATH, file_name[2:])

                # Write data
                write_data(fo, size_dist_df)

                # Write labels to file
                write_label(fo_label, labels)

    train_file, test_file = gen_scp_files()
    train_label_file, test_label_file = master_label_file()

    logging.info('Adding deltas and acelerations...')

    test_count = HCopy(['-C', CONFIG_HCOPY_FILE_PATH,
                        '-S', TEST_HCOPY_SCP,
                        '-T', 1])
    logging.info("Test files:\t" + str(test_count))

    # Train
    train_count = HCopy(['-C', CONFIG_HCOPY_FILE_PATH,
                         '-S', TRAIN_HCOPY_SCP,
                         '-T', 1])
    logging.info("Train files:\t" + str(train_count))

    X_train = {'script_file': train_file, 'count': train_count, 'id': params['data_version'] + '.train.synth.synth'}
    X_test = {'script_file': test_file, 'count': test_count, 'id': params['data_version'] + '.test.synth.synth'}

    y_train = {'mlf': train_label_file, 'count': None, 'id': params['data_version'] + '.train.synth.synth'}
    y_test = {'mlf': test_label_file, 'count': None, 'id': params['data_version'] + '.test.synth.synth'}

    return X_train, X_test, y_train, y_test